In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from superintendent import ClassLabeller
import pickle
from tqdm.notebook import tqdm

from scripts import dl_utils
from scripts.viz_tools import plot_image_grid

In [ ]:
def compute_sahm(p):
    p = p.astype(int)
    sahm = (p[:,:,11] - p[:,:,10]) / (p[:,:,11] + p[:,:,10])
    sahm[p.mask[:,:,0]] = np.nan
    return sahm

In [ ]:
rect_width = 0.01
coords = [107.97332905436846,-6.787530131094053]
start_date = '2016-06-01'
end_date = '2022-01-01'
patches, raster_info = dl_utils.download_patch(dl_utils.rect_from_point(coords, rect_width), start_date, end_date)
cloud_free = [index for index, p in enumerate(patches) if np.sum(p.mask) / np.size(p) < 0.50]
patches = [patches[i] for i in cloud_free]
raster_info = [raster_info[i] for i in cloud_free]
dates = [r['metadata']['']['id'][15:25] for r in raster_info]

In [ ]:
sahm = np.array([compute_sahm(patch) for patch in patches])
candidates = [i for i, patch in enumerate(sahm) if np.nansum(patch > 0.0) > 0]
num_img = int(np.ceil(np.sqrt(len(candidates))))
rgb_img = []
for i in candidates:
    img = np.clip(patches[i][:,:,3:0:-1] / 3000, 0, 1)
    sahm = compute_sahm(patches[i])
    sahm_img = np.copy(img)
    sahm_img[sahm > 0.0] = 1
    rgb_img.append(np.concatenate((img, sahm_img), 1))
#rgb_img = [np.clip(patches[i][:,:,3:0:-1] / 3000, 0, 1) for i in candidates]
print(len(candidates))

In [ ]:
widget = ClassLabeller.from_images(
    features=rgb_img,
    canvas_size=(1000, 500),
    options=['Fire', 'uncertain', 'No Fire']
)
widget

In [ ]:
labels = np.array(widget.new_labels)

In [ ]:
label_class = 1
fn = f"{coords[0]:.3f}-{coords[1]:.3f}_class-{label_class}"
data_index = np.array(candidates)[np.where(labels == 'Fire')[0]]
if len(data_index) > 1:
    data_to_dump = np.array(patches)[data_index]
    with open(f'../../data/training_data/fires/{fn}.pkl', 'wb') as f:
        pickle.dump(data_to_dump, f)
    with open(f'../../data/training_data/fires/{fn}_labels.pkl', 'wb') as f:
        pickle.dump([label_class] * len(data_to_dump), f)
    # View all images
    figure_file_path = f'../../data/training_data/fires/{fn}.png'
    plot_image_grid(data_to_dump, file_path=figure_file_path)

In [ ]:
label_class = 0
fn = f"{coords[0]:.3f}-{coords[1]:.3f}_class-{label_class}"
data_index = np.array(candidates)[np.where(labels == 'No Fire')[0]]
data_to_dump = np.array(patches)[data_index]
with open(f'../../data/training_data/fires/{fn}.pkl', 'wb') as f:
    pickle.dump(data_to_dump, f)
with open(f'../../data/training_data/fires/{fn}_labels.pkl', 'wb') as f:
    pickle.dump([label_class] * len(data_to_dump), f)
# View all images
figure_file_path = f'../../data/training_data/fires/{fn}.png'
plot_image_grid(data_to_dump, file_path=figure_file_path)